# **Play Tic-Tac-Toe Against a Q-Learning Agent**

Hello,

My name is **Anas HAMOUTNI** and I am a **Moroccan Data Scientist**.

**Tic Tac Toe** is a classic two-player game where one player uses '**X**' and the other uses '**O**'. The goal is to **get three of your own markers in a row**, either horizontally, vertically, or diagonally. The game ends with a winner if someone achieves this or a draw if the board is filled without any three-in-a-row.

This project implements a **console-based Tic Tac Toe game** where a human plays against an AI that learns through **[Q-learning](https://www.simplilearn.com/tutorials/machine-learning-tutorial/what-is-q-learning)**. The game includes features like **random starting turns**, **intelligent AI moves**, **scoring**, and **replay options**. All the rules of Tic Tac Toe are properly enforced, including **legal moves** and **checking for the winner**.

### **Rules**

The game follows the standard rules of **Tic Tac Toe**:

*   The game is played on a 3x3 grid.
*   The player is '**-1**', and the AI is '**1**'.
*   Players take turns to place their marks in empty squares.
*   The first player to get three of their marks in a horizontal, vertical, or diagonal row wins.

If all nine squares are filled without a winner, the game is a draw.

### **About Q-learning:**

**Q-learning** is a model-free **reinforcement learning algorithm** used to find the optimal action-selection policy for a given finite **[Markov decision process](hhttps://www.youtube.com/watch?v=2iF9PRriA7w)**.

The algorithm works by learning a **Q-function**, represented by $Q(s, a)$, where $s$ is the current state and $a$ is the chosen action.

The **Q-function** represents the expected return (**cumulative reward**) when taking action $a$ in state $s$ and following the optimal policy thereafter.

The **Q-values** are iteratively updated using the following formula:
$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left( r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right)$$

Here, $α$ is the learning rate, $r$ is the immediate reward received after taking action $a$, $γ$ is the discount factor that determines the importance of future rewards, $s'$ is the next state, and $a'$ is the next action.

The term $\max_{a'} Q(s', a')$ represents the **maximum Q-value** for the next state, which essentially captures the expected return of the best action in the next state.

**The algorithm iterates through states and actions**, continually updating **Q-values** until convergence, to ultimately obtain the optimal policy.

--------------

**Enjoy playing against the AI, and may the best player win!**

In [62]:
#@title ##**1. Import Libraries** { display-mode: "form" }

# Importing essential libraries for numerical operations and deep learning
import numpy as np
import random

In [63]:
#@title ##**2. Initialize Q-table** { display-mode: "form" }

# Initialize Q-table
Q = {}
alpha = 0.5  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.1  # Exploration rate


In [64]:
#@title ##**3. Helper Functions** { display-mode: "form" }

# Function to convert the board to a hashable string (flattening it into a one-dimensional array and then converting it to a string)
def board_to_key(board):
    return str(board.reshape(-1))

# Function to find available actions (i.e., empty cells) from the current state of the board
def available_actions(board):
    return np.argwhere(board == 0) # Returns the indices of the cells where the value is 0 (empty)

# Function to choose an action and return its index using a policy that combines exploration and exploitation
def choose_action(board):
    key = board_to_key(board) # Convert the board to a string key
    if key not in Q:
        Q[key] = np.zeros(len(available_actions(board))) # Initialize Q-values if key not present
    actions = available_actions(board) # Get available actions
    if random.random() < epsilon: # With probability epsilon, choose a random action (exploration)
        action_index = random.randint(0, len(actions) - 1)
    else:
        action_index = np.argmax(Q[key]) # Otherwise, choose the action with the highest Q-value (exploitation)
    return actions[action_index], action_index # Return the chosen action and its index

# Function to update the Q-table using the Q-learning update rule
def update_q_table(prev_state, action_index, reward, current_state):
    prev_key = board_to_key(prev_state) # Convert previous state to string key
    current_key = board_to_key(current_state) # Convert current state to string key

    if prev_key not in Q: # If previous key not present, initialize Q-values
        Q[prev_key] = np.zeros(len(available_actions(prev_state)))

    if current_key not in Q: # If current key not present, initialize Q-values
        Q[current_key] = np.zeros(len(available_actions(current_state)))

    max_q_value_next_state = np.max(Q[current_key]) if Q[current_key].size != 0 else 0 # Get the maximum Q-value for the next state

    # Apply the Q-learning update formula
    Q[prev_key][action_index] += alpha * (reward + gamma * max_q_value_next_state - Q[prev_key][action_index])


# Function to check if there's a winner (3 in a row horizontally, vertically, or diagonally)
def check_winner(board):
    for row in range(3):
        if board[row, 0] == board[row, 1] == board[row, 2] != 0:
            return True # Check horizontal
    for col in range(3):
        if board[0, col] == board[1, col] == board[2, col] != 0:
            return True # Check vertical
    if board[0, 0] == board[1, 1] == board[2, 2] != 0:
        return True # Check main diagonal
    if board[0, 2] == board[1, 1] == board[2, 0] != 0:
        return True # Check other diagonal
    return False # No winner

# Function to check if the board is full (i.e., a draw)
def is_draw(board):
    return not (board == 0).any() # Check if there are any zeros (empty cells) in the board

# Function for handling the human player's move, including input validation
def human_move(board):
    while True:
        move = input("Enter your move (0-8): ") # Get input from the user
        try:
            move = int(move) # Convert input to integer
            if move < 0 or move > 8:
                raise ValueError("Move out of range") # Check if input is within the valid range
            row, col = divmod(move, 3) # Convert the input to row and column indices
            if board[row, col] == 0:
                return row, col # Return the chosen row and column if the cell is empty
            else:
                print("Illegal move. Try again.") # If cell is not empty, prompt to try again
        except ValueError: # Handle non-integer or out-of-range inputs
            print("Invalid input. Please enter an integer between 0 and 8.")

In [65]:
#@title ##**4.  The Game Loop** { display-mode: "form" }

# Function to play Tic Tac Toe game between a user and an AI
def play():
    user_score = 0 # Initialize user's score
    ai_score = 0 # Initialize AI's score

    # Main game loop
    while True:
        board = np.zeros((3, 3)) # Initialize the game board
        turn = 'user' if random.random() < 0.5 else 'ai' # Randomly decide who goes first
        print(f"{turn.capitalize()} goes first!") # Print who is going first
        prev_state = None # Store previous state of the board
        action_index = None # Store the index of the action taken

        # Loop for each turn in the game
        while True:
            print(board) # Print the current state of the board

            # If it's the user's turn
            if turn == 'user':
                row, col = human_move(board) # Get the user's move
                board[row, col] = 1 # Update the board with the user's move
                if check_winner(board): # Check if the user has won
                    print(board) # Print the updated board
                    print("You win!") # Declare user's victory
                    user_score += 1 # Increment user's score
                    break # Exit the turn loop
                elif is_draw(board): # Check for a draw
                    print(board)
                    print("It's a draw!")
                    if prev_state is not None:
                        update_q_table(prev_state, action_index, 0, board) # Update Q-table for a draw
                    break
                turn = 'ai' # Change the turn to the AI

            # If it's the AI's turn
            else:
                action, action_index = choose_action(board) # Choose the AI's action
                row, col = tuple(action) # Extract row and column from the action
                board[row, col] = -1 # Update the board with the AI's move
                if check_winner(board): # Check if the AI has won
                    print(board)  # Print the updated board
                    print("AI wins!") # Declare AI's victory
                    ai_score += 1 # Increment AI's score
                    if prev_state is not None:
                        update_q_table(prev_state, action_index, 1, board) # Update Q-table with win
                    break
                elif is_draw(board): # Check for a draw
                    print("It's a draw!")
                    if prev_state is not None:
                        update_q_table(prev_state, action_index, 0, board) # Update Q-table for a draw
                    break
                if prev_state is not None:
                    update_q_table(prev_state, action_index, 0, board) # Update Q-table for regular move
                prev_state = board.copy() # Save the current state as the previous state for the next iteration
                turn = 'user' # Change the turn to the user
        print(f"Score: User {user_score} - AI {ai_score}") # Print the current score
        replay = input("Play again? (y/n): ") # Ask if the player wants to play again
        if replay.lower() != 'y':
            break # Exit the main game loop if the answer is not 'y'

In [66]:
#@title ##**5. Start the Game** { display-mode: "form" }

play() # Start the game

User goes first!
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Enter your move (0-8): 1
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[-1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
Enter your move (0-8): 4
[[-1.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
[[-1.  1. -1.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
Enter your move (0-8): 7
[[-1.  1. -1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]]
You win!
Score: User 1 - AI 0
Play again? (y/n): N
